# 🎬 IMDb Data Analysis – Dataset Creation Overview

This notebook outlines the process of creating a comprehensive dataset for analyzing movie and television data from IMDb.

## 📦 Data Sources

1. **IMDb Datasets**  
   Source: [https://datasets.imdbws.com/](https://datasets.imdbws.com/)  
   These datasets are updated daily and include comprehensive information about:
   - Titles (`title.basics.tsv.gz`)
   - Ratings (`title.ratings.tsv.gz`)
   - Crew and principal cast (`title.crew.tsv.gz`, `title.principals.tsv.gz`)
   - Box office and production details

2. **Box Office Mojo API (via `boxoffice_api`)**  
   This Python package allows retrieval of box office performance data, including:
   - Domestic and international grosses
   - Opening weekend performance
   - Distributor metadata

## 🛠️ Dataset Creation Workflow

- **Step 1:** Download and extract IMDb `.tsv.gz` files into structured DataFrames
- **Step 2:** Clean and filter relevant records (e.g., movies only, specific years, non-null ratings)
- **Step 3:** Enrich the dataset by calling the `boxoffice_api` to pull gross revenue and studio metadata
- **Step 4:** Merge datasets to form a unified view of titles with both IMDb ratings and box office performance
- **Step 5:** Store as a clean Delta or Parquet table for downstream analysis in Databricks or PySpark

> ⚠️ Note: Ensure compliance with IMDb's [terms of use](https://www.imdb.com/conditions) when using and distributing the dataset.

---

This dataset will serve as the foundation for analyzing trends in movie ratings, revenue, and genres with Microsoft Power BI.


In [0]:
%pip install bs4  # Install BeautifulSoup library for web scraping

In [0]:
%pip install boxoffice-api

In [0]:
# Restart the Python environment to apply changes made by library installations or updates
dbutils.library.restartPython()

In [0]:
import json  # For handling JSON data
import os  # For interacting with the operating system
import requests  # For making HTTP requests
import urllib.request  # For opening and reading URLs
from boxoffice_api import BoxOffice
from bs4 import BeautifulSoup  # For parsing HTML and XML documents
from delta.tables import DeltaTable  # For working with Delta Lake tables
from pyspark.sql import Row  # For creating Spark DataFrame rows
from pyspark.sql.functions import *

In [0]:
# Retrieve the OMDB API key from Databricks secrets
omdbkey = dbutils.secrets.get(scope = "djsdbsecrets", key = "omdbapikey")

In [0]:
ext_df = spark.sql('DESCRIBE EXTERNAL LOCATION `externaloc`')  # Retrieve metadata for the external location
ext_loc = ext_df.select('url').collect()[0][0]  # Extract the URL of the external location from the DataFrame
catalog_name = "data_analysis"  # Catalog name for the database
schema_name = "imdb_data"  # Schema name for the database
# Create a catalog if it does not exist, specifying a managed location
spark.sql("CREATE CATALOG IF NOT EXISTS {0} MANAGED LOCATION '{1}'".format(catalog_name, ext_loc))

# Create a schema within the specified catalog if it does not exist
spark.sql("CREATE SCHEMA IF NOT EXISTS {0}.{1}".format(catalog_name, schema_name))


In [0]:
download_path = '/Volumes/generaldata/dataanalysis/upload/imdb/'  # Path to download IMDB data
url = 'https://datasets.imdbws.com/'  # URL for IMDB datasets

In [0]:
html_content = requests.get(url).text  # Fetch HTML content from the specified URL
soup = BeautifulSoup(html_content, 'html.parser')  # Parse the HTML content using BeautifulSoup
items_list = soup.find('ul')  # Locate the unordered list in the parsed HTML
for item in items_list.findAll('a'):  # Iterate through all anchor tags within the list
   file_name = item.getText()  # Extract the text (file name) from the anchor tag
   decompressed_file_name = file_name.replace('.gz', '')  # Remove the .gz extension for the decompressed file name
   file_path = item.get('href')  # Get the href attribute (URL) of the anchor tag
   dest_download_path = "/tmp/{}".format(file_name)  # Define the temporary download path for the file
   urllib.request.urlretrieve(file_path, dest_download_path)  # Download the file to the temporary path
   os.system('gzip -d {}'.format(dest_download_path))  # Decompress the downloaded .gz file
   os.system("cp /tmp/{0} {1}".format(decompressed_file_name, download_path))  # Copy the decompressed file to the final download path

In [0]:
for file in dbutils.fs.ls(download_path):
    table_name = file.name.replace('.tsv', '').replace('.', '_')  # Create table name from file name
    full_table_name = f"{catalog_name}.{schema_name}.{table_name}"  # Construct full table name with catalog and schema
    if spark._jsparkSession.catalog().tableExists(full_table_name):  # Check if the table already exists
        path = file.path  # Get the file path
        df = spark.read.option("delimiter", "\t").option("header", "true").csv(path)  # Read the TSV file into a DataFrame
        df = df.replace(r"\N", None)  # Replace '\N' with None in the DataFrame
        df.write\
          .mode("overwrite")\
          .option("overwriteSchema", "true")\
          .saveAsTable("{0}.{1}.{2}".format(catalog_name, schema_name, table_name))  # Save DataFrame as a table

In [0]:
json_data = []  # Initialize an empty list to store the JSON data
box_office = BoxOffice()  # Create an instance of the BoxOffice class
for y in range(2010, 2025):  # Loop through the years from 2010 to 2023
    daily_data = box_office.get_yearly(year=y)  # Fetch daily box office data for the year
    for d in daily_data:  # Iterate through each day's data
        d['Release Date'] = d['Release Date'] + ' ' + str(y)  # Append the year to the release date
    json_data.append(daily_data)  # Add the daily data for the year to the json_data list

In [0]:
# List of movie genres to be used for filtering and analysis
genres = [
    'Action',
    'Adventure',
    'Comedy',
    'Drama',
    'Fantasy',
    'Horror',
    'Mystery',
    'Romance',
    'Science Fiction',
    'Thriller',
    'Western',
    'Animation',
    'Crime',
    'Documentary',
    'Family',
    'Musical',
    'War',
    'Historical',
    'Sports',
    'Biography'
]

In [0]:
table_name = "omdb_analysis"  # Define the name of the table to store the analysis results

json_p = spark.sparkContext.parallelize(json_data)  # Convert JSON data to an RDD
df = spark.read.json(json_p)  # Read the RDD as a DataFrame

# Clean 'Gross' and 'Total Gross' columns by removing dollar signs and commas
df = df.withColumn(
            "Gross",
            regexp_replace("Gross", "\\$", ""))\
      .withColumn(
            "Gross",
            regexp_replace("Gross", ",", ""))\
      .withColumn(
            "Total Gross",
            regexp_replace("Total Gross", "\\$", ""))\
      .withColumn(
            "Total Gross",
            regexp_replace("Total Gross", ",", ""))

# Cast 'Gross' and 'Total Gross' columns to float type
df = df.withColumn("Gross", col("Gross").cast("float"))
df = df.withColumn("Total Gross", col("Total Gross").cast("float"))

# Convert 'Release Date' to date type
df = df.withColumn("Release Date", to_date(col("Release Date"), "MMM d yyyy"))

# SQL query to fetch movie data from IMDb
imdb_query = """
      SELECT
            tb.tconst,
            tb.primaryTitle,
            tb.runtimeMinutes,
            tb.genres,
            tr.averageRating,
            tr.numVotes
      FROM data_analysis.imdb_data.title_basics AS tb
      INNER JOIN data_analysis.imdb_data.title_ratings AS tr ON tb.tconst = tr.tconst
      WHERE 
            tb.titleType = 'movie'  
            AND tb.isAdult = '0'
"""
imdb_df = spark.sql(imdb_query)  # Execute the SQL query and store the result in a DataFrame

# Split 'genres' column into an array of genres
imdb_df = imdb_df.withColumn("GenreArray", split(col("genres"), ","))
# Find intersection of GenreArray and predefined genres
imdb_df = imdb_df.withColumn(
    "Matches",
    array_intersect(
        col("GenreArray"),
        array(*[lit(g) for g in genres])  
    )
)
# Find intersection of GenreArray and predefined genres
imdb_df = imdb_df.withColumn("Primary_Genre", element_at(col("Matches"), 1))  # Get the first matching genre
cols_to_drop = ["Matches", "Genres", "GenreArray"]  # Specify columns to be dropped from the DataFrame
imdb_df = imdb_df.drop(*cols_to_drop)  # Drop unnecessary columns from the DataFrame

df = df.join(imdb_df, df.Release == imdb_df.primaryTitle, "inner")  # Join box office data with IMDb data on movie title
df = df.withColumnRenamed("Release Date", "Release_Date")\
       .withColumnRenamed("Gross", "Gross_Earnings")\
       .withColumnRenamed("Total Gross", "Total_Gross_Earnings")  # Rename columns for consistency

df.write\
  .mode("overwrite")\
  .option("mergeSchema", "true") \
  .saveAsTable("{0}.{1}.{2}".format(catalog_name, schema_name, table_name))  # Save DataFrame as a table in the specified database